# Creating cell aggregates

We want to create 500 cell aggreagates of 100 cells each. To create these groupings, we will randomly sample 500 cell from the latent space embedding. Using a nearest neighbor search we will then find the 100 nearest neighbors of these cells. Any group of cells which has an overlap >80% with any of the previously created groups will be removed. These cell aggregates can then be used for downstream correlation tasks.

In [2]:
import scanpy as sc
import scvi
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import seaborn as sns
from scipy import sparse
import seaborn as sns
import pandas as pd

Global seed set to 0


In [4]:
# read in the anndata object which contains the latent space embedding
adata = scvi.data.read_h5ad("gpu_trained_20_dim/anndata_object")

# read in ArchR gene expression matrix


# read in ArchR gene score matrix

# subset the dataset to contain only cells which we also find in the ArchR gene expression matrix

In [7]:
# read in gene_expression matrix from ArchR
archr_gene_expr = scvi.data.read_h5ad("ArchR_gene_expr.h5ad")

In [4]:
# read in gene_scores based on scATAC-seq from ArchR
archr_gene_scores = pd.read_csv("archr_gene_scores_table.csv", sep = " ", header=0)
#archr_gene_scores = scvi.data.read_h5ad("ArchR_gene_scores.h5ad")
#archr_gene_scores_p2g = scvi.data.read_h5ad("ArchR_gene_scores_p2g.h5ad")

In [5]:
archr_gene_scores

,E8.5_rep1#TTACGTTTCTGGCATG-1,E8.5_rep1#TCCTCTAAGTCCTTCA-1,E8.5_rep1#TATCCAGCACAGACTC-1,E8.5_rep1#GAGAACCAGACACTTA-1,E8.5_rep1#TCAAGTATCTTAGCGG-1,E8.5_rep1#CCCTTAATCTCACAAA-1,E8.5_rep1#TGACCAAGTTGGTGAC-1,E8.5_rep1#TGATGACTCAAGGACA-1,E8.5_rep1#GGTAAGGGTTTACGTC-1,E8.5_rep1#CTGAATATCATCCTCA-1,...,E8.0_rep2#GACATTATCTATCGCC-1,E8.0_rep2#TCGTAATCATTCAGCA-1,E8.0_rep2#CGCAATAGTAGCTAAT-1,E8.0_rep2#TTGTGAGGTCAGGCCA-1,E8.0_rep2#TGCACCTTCATAAGCC-1,E8.0_rep2#TCATACTTCAAGACTC-1,E8.0_rep2#TGCACACCAATTAGCT-1,E8.0_rep2#GGCAAGCCAGCCTTGG-1,E8.0_rep2#AGTTACATCGTGCTAG-1,E8.0_rep2#GCTAATATCAAACCTA-1
Xkr4,0.669,0.342,0.533,0.495,0.283,0.496,0.052,0.277,0.560,0.378,...,0.0,1.874,1.952,0.0,0.0,0.0,0.000,0.0,0.0,0.0
Rp1,0.492,0.179,0.279,0.305,0.258,0.066,0.058,0.190,0.160,0.229,...,0.0,0.000,0.807,0.0,0.0,0.0,0.000,0.0,0.0,0.0
Sox17,0.732,0.341,1.093,0.958,0.386,0.564,0.175,0.540,1.197,0.440,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
Mrpl15,0.433,0.374,0.598,0.513,0.365,0.433,0.244,0.637,0.274,0.595,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
Lypla1,0.409,0.063,0.335,0.274,0.246,0.181,0.255,0.346,0.745,0.369,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hccs,0.218,0.111,0.353,0.208,0.379,0.178,0.064,0.000,0.065,0.000,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
Gm15246,0.209,0.196,0.189,0.189,0.313,0.000,0.069,0.112,0.019,0.091,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
Mid1,0.473,0.463,0.239,0.284,0.448,0.876,0.426,0.521,0.667,0.393,...,0.0,0.000,0.000,0.0,0.0,0.0,1.954,0.0,0.0,0.0
4933400A11Rik,0.212,0.165,0.077,0.191,0.236,0.499,0.379,0.168,0.603,0.127,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.0


In [8]:
# subset gene scores matrix to only contain genes for which we also have gene expression information
scores = archr_gene_scores[archr_gene_scores.index.isin(archr_gene_expr.var["name"])]

In [ ]:
scores.shape

In [9]:
scores_mat = scores.to_numpy()

In [20]:
scores_mat_sparse = sparse.csr_matrix(scores_mat)

MemoryError: Unable to allocate 8.14 GiB for an array with shape (546058225, 2) and data type int64

In [21]:
del scores_mat_sparse
gc.collect()

NameError: name 'scores_mat_sparse' is not defined

In [10]:
import h5py

https://stackoverflow.com/questions/20928136/input-and-output-numpy-arrays-to-h5py

In [11]:
h5f = h5py.File('gene_scores.h5', 'w')

In [15]:
import gc
del archr_gene_expr
del archr_gene_scores
gc.collect()
del scores
gc.collect()

NameError: name 'archr_gene_expr' is not defined

In [16]:
del scores
gc.collect()

105

In [17]:
h5f.create_dataset('gene_scores', data=scores_mat)

MemoryError: Unable to allocate 7.63 GiB for an array with shape (22273, 45986) and data type float64

In [22]:
help(h5py.File)

Help on class File in module h5py._hl.files:

class File(h5py._hl.group.Group)
 |  File(name, mode='r', driver=None, libver=None, userblock_size=None, swmr=False, rdcc_nslots=None, rdcc_nbytes=None, rdcc_w0=None, track_order=None, fs_strategy=None, fs_persist=False, fs_threshold=1, fs_page_size=None, page_buf_size=None, min_meta_keep=0, min_raw_keep=0, locking=None, **kwds)
 |  
 |  Represents an HDF5 file.
 |  
 |  Method resolution order:
 |      File
 |      h5py._hl.group.Group
 |      h5py._hl.base.HLObject
 |      h5py._hl.base.CommonStateObject
 |      h5py._hl.base.MutableMappingHDF5
 |      h5py._hl.base.MappingHDF5
 |      collections.abc.MutableMapping
 |      collections.abc.Mapping
 |      collections.abc.Collection
 |      collections.abc.Sized
 |      collections.abc.Iterable
 |      collections.abc.Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __init__(self, name, mode='r', driver=None, 

In [ ]:
h5f.close()

In [ ]:
scores_mat.shape

In [ ]:
scores_mat.shape

In [ ]:
#archr_gene_scores_p2g = scvi.data.read_h5ad("ArchR_gene_scores_p2g.h5ad")

In [ ]:
# subset anndata object to contain only overlapping cells
latent_adata = adata[archr_gene_expr.obs.index, :]

In [ ]:
latent = latent_adata.obsm["X_scVI"]

### Sample cells

Since we have 45,991 cells in our dataset, we will sample 1000 cell aggregates of 50 cells each. 

In [ ]:
class Cell_aggregates:
    def 

In [ ]:
n_aggregates = 1000 
# from all cells, randomly sample 500
sample_cells = np.random.choice(latent.shape[0], n_aggregates, replace=False)

In the following we want to find the 100 nearest neighbor for each sampled cell. We will restrict the nearest neighbors to only come from the same celltype. Since some celltypes contain less than 100 cells we will have to simply use all cells of this celltype and group them together. 

In [ ]:
metadata = latent_adata.obs
# create column for cell names
metadata["cells"] = metadata.index
# create index for cells
metadata["idx"] = np.arange(len(metadata))

## Compute k nearest neighbors

In [ ]:
k = 50
nbrs = NearestNeighbors(n_neighbors=k, algorithm="ball_tree").fit(latent)

In [ ]:
distances, indices = nbrs.kneighbors(latent)

### Filter for only the sampled cells

In [ ]:
# subset the nearest neighbors to only contain the sampled cells
filt_dist = distances[sample_cells, ]
filt_ind = indices[sample_cells, ]
print(filt_dist.shape, filt_ind.shape)

In [ ]:
@jit(nopython=True)
def check_overlap(mtx):
    nrow = mtx.shape[0]
    ncol = mtx.shape[1]
    # create an array to store whether a cell passed the overlap check
    # all entries are initially False
    considered_cells = np.zeros(nrow).astype(np.bool8)
    
    # loop over each cell and cosider to add it to the set of cell aggregates
    for i in range(nrow):
        check = True
        # loop over all previous aggregates 
        for comp in np.where(considered_cells)[0]:
            # get the number of cells which overlap between the neighborhood of the cell we would like to add and the neighborhoud of previous cell "comp"
            intersect = np.intersect1d(mtx[i, :], mtx[comp, :])
            # for each comparison between current cell i which we would like to add and previous cell which we are comparing to
            # compute the percentage of overlap
            if (len(intersect) / ncol) > 0.8: # if the intersection is above 0.8, we do not consider it
                check = False
                break
        if check:
            considered_cells[i] = True
    # get indices
    return np.arange(start=0, stop=nrow, step=1)[considered_cells]

In [ ]:
idx_keep = check_overlap(filt_ind) # the first row corresponds to the sampled cell itself

Filter the neigbors for cells which passed the overlap check

In [ ]:
aggregates_keep = filt_ind[idx_keep, :]

#### Plot cells which pass overlap filtering and not

In [ ]:
#subset_df = metadata[metadata.idx.isin(filt_ind[:, 0])]

In [ ]:
#subset_df["pass_overlap"] = np.where(subset_df["idx"].isin(test[:,0]), True, False)

In [ ]:
#import seaborn as sns

In [ ]:
#sns.barplot(x = "celltype", y = "pass_overlap", hue = "pass_overlap", data = subset_df)

## Only keep cells of same celltype

In [ ]:
def filter_celltypes(mat, metadata):
    groups={}
    # check whether a cell aggregate contains cells of other celltypes and remove then
    for n, i in enumerate(sample_cells):
        # get celltype information of sampled cell
        celltype_test_cell = metadata.iloc[i]["celltype"]
        # get indices of cells which are in the neighborhood 
        neighbor_cells = mat[mat[:, 0] == i, :]
        # get cells which are of the same celltype, vector includes the sampled cell itself
        keep = np.array(metadata[(metadata.idx.isin(neighbor_cells.flatten())) & (metadata.celltype == celltype_test_cell)]["idx"])
        # keep only aggregates which contain at least 10 cells after removing non-matching celltypes
        if keep.shape[0] > 10:
            groups[i] = keep
        else:
            continue
    return groups
    

In [ ]:
metadata = latent_adata.obs
# create column for cell names
metadata["cells"] = metadata.index
# create index for cells
metadata["idx"] = np.arange(len(metadata))

In [ ]:
groups = filter_celltypes(aggregates_keep, metadata)

In [ ]:
len(groups)

## Aggregate expression/scores/accessibility 

In [ ]:
rna_mat = archr_gene_expr.X
rna_mat.shape

In [ ]:
rna_mat

In [ ]:
rna_mat[0:10, 0:10].todense()

TODO: Implement aggregation with numba -> convert to csr matrx

In [ ]:
def create_aggregates(mat, groups):
    # initialize matrix to store average gene expression for each cell aggregate
    # the matrix has dimensions genes x cell aggregates
    rna_agg = np.zeros((mat.shape[1],len(groups)))
    # for each cell aggregate calculate the average gene expression for each gene
    for i, g in enumerate(groups):
        rna_agg[:, i] = mat[groups[g], :].mean(axis=0).flatten()
    return sparse.csr_matrix(rna_agg) # return sparse matrix genes x cell aggregates

In [ ]:
# gene expression
rna_agg = create_aggregates(rna_mat, groups)

In [ ]:
rna_agg.shape

In [ ]:
rna_agg

In [ ]:
rna_mat.shape

In [ ]:
scores_mat.shape

In [ ]:
scores_mat.T.shape

In [ ]:
# gene scores
score_agg = create_aggregates(scores_mat.T, groups)

In [ ]:
score_agg.shape

In [ ]:
score_agg[1:50, 1:50].todense()

## Correlations

To compute row-wise correlations:

In [ ]:
A = rna_agg.copy()
print(A.shape)
print(np.mean(A, axis=1).shape)
A -= np.mean(A, axis=1)
print(A.shape)
print(np.std(A, axis=1).shape)
print(np.std(A, axis=1))
#A /= np.std(A, axis=1)


In [ ]:
A[np.where(np.std(A, axis=1) == 0)].shape

In [ ]:
A = rna_agg.copy()
A.shape
A -= np.mean(A, axis=1)
A.shape
A /= np.std(A, axis=1)
B = score_agg.copy()

B -= np.mean(B, axis=1)
B /= np.std(B, axis=1)

corr = np.mean(A*B, axis = 1)

In [ ]:
corr

In [ ]:
A = rna_agg.copy()
B = score_agg.copy()
cA = A - A.mean(axis=1)
cB = B - B.mean(axis=1)
print(cA.shape)
print(np.square(cA))
#print((cA**2).shape)
sA = np.sqrt((np.square(cA)).mean(axis=1))
sB = np.sqrt((cB**2).mean(axis=1))
corr = (cA*cB).mean(axis=1) / sA*sB

In [ ]:
corr

In [ ]:
aggregate_idx = filt_ind[indices[sampled_cells, 0] == test, :]

In [ ]:
aggregate_idx.flatten()

In [ ]:
celltype = metadata.iloc[test]["celltype"]

In [ ]:
subset = np.array(metadata[(metadata.idx.isin(aggregate_idx.flatten())) & (metadata.celltype == celltype)]["idx"])
subset
#subset[subset["celltype"] == metadata.iloc[test]["celltype"]]["idx"]

In [ ]:
n_aggregates = 500
n_cells_per_aggregate = 100
group_mat = np.zeros([n_aggregates, n_cells_per_aggregate])
celltype_list = []

for n, i in enumerate(sample_cells):
    test_cell = sample_cells[n]
    # get celltype information for the cell
    celltype_test_cell = metadata.iloc[test_cell]["celltype"]
    celltype_list.append(celltype_test_cell)

    # get all other cells which are of the same celltype and convert to numpy array
    same_celltype = np.array(metadata[metadata["celltype"] == celltype_test_cell]["idx"])
    
    # First we will check if there are more than 100 cells of this celltype
    if same_celltype.shape[0] >= n_cells_per_aggregate:
        #print("There are more than 100 cells of this celltype present")
        # from the connectivity matrix get only the columns which correspond to cells of the same celltype
        subset_conn = connectivities[test_cell, same_celltype]
        subset_dist = distances[test_cell, same_celltype]
        # get the 100 cells with the smallest distance
        top_hundred = np.argpartition(subset_dist.todense(), n_cells_per_aggregate)[:, :n_cells_per_aggregate]

        # subset the dataframe to get only cells which are nearest neighbors 
        group_idx = top_hundred.reshape([n_cells_per_aggregate,1]).flatten()
        group_cells = np.array(metadata[metadata["idx"].isin(top_hundred.tolist()[0])]["cells"])

    else:
        print(f"The number of cells for {celltype_test_cell} is below 100")
        group_idx = np.array(metadata[metadata["celltype"] == celltype_test_cell]["idx"]).flatten()
        group_cells = np.array(metadata[metadata["celltype"] == celltype_test_cell]["cells"])
    #print(group_idx.shape)
    # add the cells to the matrix at position i, to create group i
    group_mat[n, : group_cells.shape[0]] = group_idx
                

In [ ]:
group_mat

In [ ]:
np.unique(np.array(celltype_list))

In [ ]:
len(adata.obs[adata.obs["celltype"] == "PGC"])

In [ ]:
np.unique(np.array(celltype_list)).shape

In [ ]:
np.unique(np.array(celltype_list)).tolist()

In [ ]:
adata.obs.celltype.unique()

In [ ]:
group_mat

In [ ]:
test_cell = sample_cells[0]

# get celltype information for the cell
celltype_test_cell = metadata.iloc[test_cell]["celltype"]

# get all other cells which are of the same celltype and convert to numpy array
same_celltype = np.array(metadata[metadata["celltype"] == celltype_test_cell]["idx"])

In [ ]:
# First we will check if there are more than 100 cells of this celltype
if same_celltype.shape[0] >= 100:
    print("There are more than 100 cells of this celltype present")
    # from the connectivity matrix get only the columns which correspond to cells of the same celltype
    subset_conn = connectivities[test_cell, same_celltype]
    subset_dist = distances[test_cell, same_celltype]
    # get the 100 cells with the smallest distance
    top_hundred = np.argpartition(subset_dist.todense(), 100)[:, :100]

    # subset the dataframe to get only cells which are nearest neighbors 
    group_idx = top_hundred.reshape([100,1])
    group_cells = np.array(metadata[metadata["idx"].isin(top_hundred.tolist()[0])]["cells"])

else:
    print("The number of cells for this celltype is below 100")
    group_idx = np.array(metadata[metadata["celltype"] == celltype_test_cell]["idx"])
    group_cells = np.array(metadata[metadata["celltype"] == celltype_test_cell]["cells"]
                         


In [ ]:
# from the connectivity matrix get only the columns which correspond to cells of the same celltype
subset_conn = connectivities[test_cell, same_celltype]
subset_dist = distances[test_cell, same_celltype]
# get the 100 cells with the smallest distance
top_hundred = np.argpartition(subset_dist.todense(), 100)[:, :100]

# subset the dataframe to get only cells which are nearest neighbors 
group = np.array(metadata[metadata["idx"].isin(top_hundred.tolist()[0])]["cells"])

## Leiden Clustering

Since we already have celltype annotations we might simply use Leiden Clustering to create cell aggregates. We would expect these to be celltype-specific if done at high enough resolution, but we could also try to do Leiden Clustering based on each celltype individually.

In [ ]:
import igraph as ig
import leidenalg as la

With a resolution of 100, I get 930 clusters, which is probably too much. 

In [ ]:
sc.tl.leiden(adata, resolution=100, key_added="cluster_pvi")#, restrict_to=("celltype", adata.obs.celltype.unique()))

In [ ]:
adata.obs.cluster_pvi.unique()

In [ ]:
plt.hist(adata.obs.cluster_pvi, bins = 500)
plt.show()

If I reduce the resolution to 50, I get 483 cells. 

In [ ]:
sc.tl.leiden(adata, resolution=50, key_added="cluster_pvi")#, restrict_to=("celltype", adata.obs.celltype.unique()))

In [ ]:
adata.obs.cluster_pvi.unique()

In [ ]:
plt.hist(adata.obs.cluster_pvi, bins = 500)
plt.show()

In [ ]:
adata.obs.cluster_pvi

In [ ]:
adata.obs.groupby("cluster_pvi").count()

### Subset to do Leiden Clustering on individual celltypes

Does it make sense to create clusters of different size or should I aim to create clusters which are similar in size for each celltype. This means that if there are more cells of a celltype I will get more clusters of that type.

### Erythroids

In [ ]:
adata_subset = adata[adata.obs['celltype'] == "Erythroid2"]
print(f"The number of celsl for Erythroids are: {adata_subset.shape[0]}")

In [ ]:
sc.tl.leiden(adata_subset, resolution=.6, key_added="Erythroid_clusters")

In [ ]:
sc.tl.umap(adata_subset, min_dist=0.2)
sc.pl.umap(adata_subset, color='Erythroid_clusters')

In [ ]:
adata_subset.obs.groupby("Erythroid_clusters").count()

### Erythroid1

In [ ]:
adata_subset = adata[adata.obs['celltype'] == "Erythroid1"]
print(f"The number of celsl for Erythroids are: {adata_subset.shape[0]}")

In [ ]:
sc.tl.leiden(adata_subset, resolution=.6, key_added="Erythroid_clusters")

In [ ]:
sc.tl.umap(adata_subset, min_dist=0.2)
sc.pl.umap(adata_subset, color='Erythroid_clusters')

In [ ]:
adata_subset.obs.groupby("Erythroid_clusters").count()

Here I will remove Cluster 6, because it only contains 6 cells.

### Erythroid 3

In [ ]:
adata_subset = adata[adata.obs['celltype'] == "Erythroid3"]
print(f"The number of celsl for Erythroids are: {adata_subset.shape[0]}")

In [ ]:
sc.tl.leiden(adata_subset, resolution=.2, key_added="Erythroid_clusters")

In [ ]:
sc.tl.umap(adata_subset, min_dist=0.2)
sc.pl.umap(adata_subset, color='Erythroid_clusters')

In [ ]:
adata_subset.obs.groupby("Erythroid_clusters").count()

### Parietal Endoderm

In [ ]:
adata_subset = adata[adata.obs['celltype'] == "Parietal_endoderm"]
print(f"The number of celsl for Erythroids are: {adata_subset.shape[0]}")

In [ ]:
sc.tl.leiden(adata_subset, resolution=.6, key_added="clusters")

In [ ]:
sc.tl.umap(adata_subset, min_dist=0.2)
sc.pl.umap(adata_subset, color='clusters')

In [ ]:
adata_subset.obs.groupby("clusters").count()

Here I want to remove cluster 3 and cluster 4, because they contain very few cells.

In [ ]:
adata.obs.celltype.unique()

### Manus version of doing celltype-specific Leiden clustering

In [ ]:
df_leiden_list=[]

In [ ]:
for cell_type in adata.obs['celltype'].unique():
    adata_celltype =adata[adata.obs['celltype']==cell_type,:]
    if adata_celltype.shape[0]>80:
        sc.pp.neighbors(adata_celltype, use_rep="X_scVI",n_neighbors=100, n_pcs=50)
        sc.tl.leiden(adata_celltype, resolution=1)
        #sc.tl.leiden(adata_celltype, resolution=0.2)  # decreasing resolution
        sc.tl.umap(adata_celltype, spread=1., min_dist=.5, random_state=11)
        sc.pl.umap(adata_celltype, color="leiden", legend_loc="on data",edges=False,title=cell_type)
        adata_celltype.obs['leiden_name'] = [str(s) + '_'+ cell_type for s in adata_celltype.obs['leiden'] ]
        adata_celltype.obs['cell_name'] = adata_celltype.obs.index

        cluster_celltype = adata_celltype.obs[['cell_name','leiden_name']]
        df_leiden_list.append(cluster_celltype)
        
    else:
        adata_celltype.obs['leiden_name'] = [str(0) + '_'+ cell_type for s in range(adata_celltype.obs.shape[0]) ]
        adata_celltype.obs['cell_name'] = adata_celltype.obs.index

        cluster_celltype = adata_celltype.obs[['cell_name','leiden_name']]
        df_leiden_list.append(cluster_celltype)

In [ ]:
adata

In [ ]:
# higher resolution values lead to more clusters
help(sc.tl.leiden)

In [ ]:
la.find_partition(latent, la.ModularityVertexPartition)

In [ ]:
help(la.find_partition)